In [ ]:
####################################
# ring nns with single layer (1,), (2,), (5,)
####################################


import numpy as np
import matplotlib.pyplot as plt
from math import pi
from nn import Input
from tensor import RingTensor
from optimizer import SGD
from typing import cast, Callable

x = np.linspace(-1, 1, 100)
x_ = RingTensor(x[:,None])
y = np.cos(x*pi)
# y = x * 0
y = np.sin(6 * x) * np.cos(3 * pi * x) + 0.2 * x**3 - 0.1 * np.sign(x)  # interesting nontrivial function


n_neurons_list = [1, 2, 5]
fig, axs = plt.subplots(3, len(n_neurons_list), figsize=(10, 8), squeeze=False)

for run, n_neurons in enumerate(n_neurons_list):
    # initialize model and optimizer
    nn = Input((1,1)).ff(n_neurons).mean(axis=1, keepdims=True)
    opt = SGD(nn, lr=0.1, lr_decay=1.)
    nn = cast(Callable[[RingTensor], RingTensor], nn)

    # initialize history tracking variables
    ys = [np.zeros_like(y)]
    ys += [nn(x_).real().data]
    losses_history = []
    grad_history = []
    differences = []

    for i in range(50):
        # check if we're done
        # if (ys[-1] - ys[-2]).abs().sum() < 1e-2: break

        # extract individual gradients
        tmp = []
        for i in range(len(x)):
            out = nn(x_)
            assert isinstance(out, RingTensor)
            pred = out.real().squeeze(1)
            losses = ((pred - y)**2)
            losses[i].backward()
            tmp.append(nn.weights[0]._grad.tolist()[0][0])
            nn.weights[0].reset_grad()
        grad_history.append(tmp)

        # normal optimization step
        out = nn(x_)
        assert isinstance(out, RingTensor)
        pred = out.real().squeeze(1)
        losses = ((pred - y)**2)
        loss = losses.mean()
        losses_history.append(losses.data)
        loss.backward()
        opt()

        # update predictions
        ys += nn(x_).real().data,
        differences.append((ys[-1] - ys[-2]).abs().sum())



    # plot results
    axs[0, run].set_title(f"#neurons: {n_neurons}")
    ys = ys[1:]
    axs[0, run].plot(x, y, color='black')
    for i, y2 in enumerate(ys):
        color = plt.cm.autumn_r(i/len(ys))
        axs[0, run].plot(x, y2, color=color)
    if run == 0:
        axs[0, run].set_ylabel("y")
    else:
        axs[0, run].set_yticks([])
    axs[0, run].set_xticks([])
    axs[0, run].set_ylim(-1.1, 1.1)

    for i, losses in enumerate(losses_history):
        axs[1, run].plot(x, losses, color=plt.cm.autumn_r(i/len(losses_history)))
    if run == 0:
        axs[1, run].set_ylabel("loss")
    else:
        axs[1, run].set_yticks([])
    axs[1, run].set_xticks([])
    axs[1, run].set_ylim(0, 5)

    axs[2, run].plot(x, np.zeros_like(x), color='gray', linestyle='--')
    for i, grad in enumerate(grad_history):
        axs[2, run].plot(x, grad, color=plt.cm.autumn_r(i/len(grad_history)))
    if run == 0:
        axs[2, run].set_ylabel("grad (weight 0)")
    else:
        axs[2, run].set_yticks([])
    axs[2, run].set_xlabel("x")
    axs[2, run].set_ylim(-4, 4)

plt.tight_layout()
plt.show()

# print(np.array(grad_history).mean(axis=1))

In [ ]:
####################################
# just plotting code
####################################


# plot results
ys = ys[1:]
print(len(ys))
plt.plot(x, y)
for i, y2 in enumerate(ys):
    color = plt.cm.YlOrRd(i/len(ys))
    plt.plot(x, y2, color=color)
plt.title("predictions")
plt.xlabel("x")
plt.ylabel("y")
plt.show()

for i, losses in enumerate(losses_history):
    plt.plot(x, losses, color=plt.cm.YlOrRd(i/len(losses_history)))
plt.title("losses")
plt.xlabel("x")
plt.ylabel("loss")
plt.show()

plt.plot(differences, c='black')
plt.scatter(range(len(differences)), differences, c=[plt.cm.YlOrRd(i/len(differences)) for i in range(len(differences))])
plt.title("differences")
plt.xlabel("iteration")
plt.ylabel("difference")
plt.show()

for i, grad in enumerate(grad_history):
    plt.plot(x, grad, color=plt.cm.YlOrRd(i/len(grad_history)))
plt.title("grads")
plt.xlabel("x")
plt.ylabel("grad")
plt.show()

print(np.array(grad_history).mean(axis=1))

In [ ]:
####################################
# ring nn with single layer (5,)
####################################

import numpy as np
import matplotlib.pyplot as plt
from math import pi
from nn import Input
from tensor import RingTensor
from optimizer import SGD
from typing import cast, Callable

x = np.linspace(-1, 1, 100)
x_ = RingTensor(x[:,None])
y = np.cos(x*pi)
# y = x * 0
y = np.sin(6 * x) * np.cos(3 * pi * x) + 0.2 * x**3 - 0.1 * np.sign(x)  # interesting nontrivial function


n_neurons = 5
fig, axs = plt.subplots(3, n_neurons+1, figsize=(3*(n_neurons+1), 8), squeeze=False)

# initialize model and optimizer
nn_separate = Input((1,1)).ff(n_neurons)
nn = lambda x: nn_separate(x).mean(axis=1, keepdims=True)
opt = SGD(nn_separate, lr=0.1, lr_decay=1.)
nn = cast(Callable[[RingTensor], RingTensor], nn)

# initialize history tracking variables
ys = [np.zeros_like(y)]
ys += [nn(x_).real().data]
ys_separate = []
losses_history = []
losses_history_separate = []
grad_history = []
differences = []

for i in range(50):
    # check if we're done
    # if (ys[-1] - ys[-2]).abs().sum() < 1e-2: break

    # extract individual gradients
    grad_tmp = []
    losses_tmp = []
    for i in range(len(x)):
        out = nn(x_)
        assert isinstance(out, RingTensor)
        pred = out.real().squeeze(1)
        losses = ((pred - y)**2)
        losses[i].backward()
        losses_tmp.append(losses[i].data.item())
        grad_tmp.append(nn_separate.weights[0]._grad.tolist()[0])
        nn_separate.weights[0].reset_grad()
    grad_history.append(grad_tmp)
    losses_history_separate.append(losses_tmp)

    # normal optimization step
    out = nn(x_)
    assert isinstance(out, RingTensor)
    pred = out.real().squeeze(1)
    losses = ((pred - y)**2)
    loss = losses.mean()
    losses_history.append(losses.data)
    loss.backward()
    opt()

    # update predictions
    ys += nn(x_).real().data,
    ys_separate += nn_separate(x_).real().data,
    differences.append((ys[-1] - ys[-2]).abs().sum())



# plot results
axs[0, 0].set_title(f"All neurons")
ys = ys[1:]
axs[0, 0].plot(x, y, color='black')
for i, y2 in enumerate(ys):
    color = plt.cm.autumn_r(i/len(ys))
    axs[0, 0].plot(x, y2, color=color)
if 0 == 0:
    axs[0, 0].set_ylabel("y")
else:
    axs[0, 0].set_yticks([])
axs[0, 0].set_xticks([])
axs[0, 0].set_ylim(-1.1, 1.1)

for i, losses in enumerate(losses_history):
    axs[1, 0].plot(x, losses, color=plt.cm.autumn_r(i/len(losses_history)))
if 0 == 0:
    axs[1, 0].set_ylabel("loss")
else:
    axs[1, 0].set_yticks([])
axs[1, 0].set_xticks([])
axs[1, 0].set_ylim(0, 3)

axs[2, 0].plot(x, np.zeros_like(x), color='gray', linestyle='--')
for i, grad in enumerate(grad_history):
    axs[2, 0].plot(x, np.mean(grad, axis=1), color=plt.cm.autumn_r(i/len(grad_history)))
if 0 == 0:
    axs[2, 0].set_ylabel("grad (weight 0)")
else:
    axs[2, 0].set_yticks([])
axs[2, 0].set_xlabel("x")
axs[2, 0].set_ylim(-2.1, 2.1)


# plot results for each neuron
for neuron in range(1, n_neurons+1):
    axs[0, neuron].set_title(f"#neuron: {neuron}")
    axs[0, neuron].plot(x, y, color='black')
    for i, y2 in enumerate(ys_separate):
        color = plt.cm.autumn_r(i/len(ys_separate))
        axs[0, neuron].plot(x, y2[:,neuron-1], color=color)
    if neuron == 0:
        axs[0, neuron].set_ylabel("y")
    else:
        axs[0, neuron].set_yticks([])
    axs[0, neuron].set_xticks([])
    axs[0, neuron].set_ylim(-1.1, 1.1)

    # for i, losses in enumerate(losses_history_separate):
    #     axs[1, neuron].plot(x, losses[neuron-1], color=plt.cm.autumn_r(i/len(losses_history_separate)))
    if neuron == 0:
        axs[1, neuron].set_ylabel("loss")
    else:
        axs[1, neuron].set_yticks([])
    axs[1, neuron].set_xticks([])
    axs[1, neuron].set_ylim(0, 3)

    axs[2, neuron].plot(x, np.zeros_like(x), color='gray', linestyle='--')
    for i, grad in enumerate(grad_history):
        axs[2, neuron].plot(x, [g[neuron-1] for g in grad], color=plt.cm.autumn_r(i/len(grad_history)))
    if neuron == 0:
        axs[2, neuron].set_ylabel("grad (weight 0)")
    else:
        axs[2, neuron].set_yticks([])
    axs[2, neuron].set_xlabel("x")
    axs[2, neuron].set_ylim(-2.1, 2.1)

plt.tight_layout()
plt.show()

# print(np.array(grad_history).mean(axis=1))

In [ ]:
####################################
# ring nn with two layers (5, 1)
####################################


import numpy as np
import matplotlib.pyplot as plt
from math import pi
from nn import Input, RingFF, Sequential
from tensor import RingTensor
from optimizer import SGD
from typing import cast, Callable

n = 100
x = np.linspace(-1, 1, n)
x_ = RingTensor(x[:,None])
y = np.cos(x*pi)
# y = x * 0
y = np.sin(6 * x) * np.cos(3 * pi * x) + 0.2 * x**3 - 0.1 * np.sign(x)  # interesting nontrivial function


n_neurons = 5
fig, axs = plt.subplots(3, n_neurons+2, figsize=(3*(n_neurons+2), 8), squeeze=False)

# initialize model and optimizer
layer1 = RingFF(1, n_neurons)
layer2 = RingFF(n_neurons, 1)
nn = Sequential([layer1, layer2])
opt = SGD(nn, lr=0.01, lr_decay=1.)
nn = cast(Callable[[RingTensor], RingTensor], nn)

# initialize history tracking variables
ys = [np.zeros_like(y)]
ys += [nn(x_).real().data]
ys_separate = []
losses_history = []
losses_history_separate = []
grad_history = []
differences = []

for i in range(200):
    # check if we're done
    # if (ys[-1] - ys[-2]).abs().sum() < 1e-2: break

    # extract individual gradients
    grad_tmp = []
    losses_tmp = []
    for i in range(n):
        out = nn(x_)
        assert isinstance(out, RingTensor)
        pred = out.real().squeeze(1)
        losses = ((pred - y)**2)
        losses[i].backward()
        losses_tmp.append(losses[i].data.item())
        grad_tmp.append(layer1.weights[0]._grad.tolist()[0])
        layer1.weights[0].reset_grad()
    grad_history.append(grad_tmp)
    losses_history_separate.append(losses_tmp)

    # normal optimization step
    out = nn(x_)
    assert isinstance(out, RingTensor)
    pred = out.real().squeeze(1)
    losses = ((pred - y)**2)
    loss = losses.mean()
    losses_history.append(losses.data)
    loss.backward()
    opt()

    # update predictions
    ys += nn(x_).real().data,
    ys_separate += layer1(x_).real().data,
    differences.append((ys[-1] - ys[-2]).abs().sum())



# plot results
axs[0, 0].set_title(f"All neurons")
ys = ys[1:]
axs[0, 0].plot(x, y, color='black')
for i, y2 in enumerate(ys):
    color = plt.cm.autumn_r(i/n)
    axs[0, 0].plot(x, y2, color=color)
if 0 == 0:
    axs[0, 0].set_ylabel("y")
else:
    axs[0, 0].set_yticks([])
axs[0, 0].set_xticks([])
axs[0, 0].set_ylim(-1.1, 1.1)

for i, losses in enumerate(losses_history):
    axs[1, 0].plot(x, losses, color=plt.cm.autumn_r(i/n))
if 0 == 0:
    axs[1, 0].set_ylabel("loss")
else:
    axs[1, 0].set_yticks([])
axs[1, 0].set_xticks([])
axs[1, 0].set_ylim(0, 3)

axs[2, 0].plot(x, np.zeros_like(x), color='gray', linestyle='--')
# for i, grad in enumerate(grad_history):
#     axs[2, 0].plot(x, np.mean(grad, axis=1), color=plt.cm.autumn_r(i/n))
if 0 == 0:
    axs[2, 0].set_ylabel("grad")
else:
    axs[2, 0].set_yticks([])
axs[2, 0].set_xlabel("x")
axs[2, 0].set_ylim(-2.1, 2.1)


# plot final neuron
for neuron in range(1,2):
    axs[0, neuron].set_title(f"#neuron: 1.{neuron}")
    axs[0, neuron].plot(x, y, color='black')
    for i, y2 in enumerate(ys):
        color = plt.cm.autumn_r(i/n)
        axs[0, neuron].plot(x, y2[:,neuron-1], color=color)
    if neuron == 0:
        axs[0, neuron].set_ylabel("y")
    else:
        axs[0, neuron].set_yticks([])
    axs[0, neuron].set_xticks([])
    axs[0, neuron].set_ylim(-1.1, 1.1)

    for i, losses in enumerate(losses_history):
        axs[1, neuron].plot(x, losses, color=plt.cm.autumn_r(i/n))
    if neuron == 0:
        axs[1, neuron].set_ylabel("loss")
    else:
        axs[1, neuron].set_yticks([])
    axs[1, neuron].set_xticks([])
    axs[1, neuron].set_ylim(0, 3)

    axs[2, neuron].plot(x, np.zeros_like(x), color='gray', linestyle='--')
    for i, grad in enumerate(grad_history):
        axs[2, neuron].plot(x, np.mean(grad, axis=1), color=plt.cm.autumn_r(i/n))
    if neuron == 0:
        axs[2, neuron].set_ylabel("grad")
    else:
        axs[2, neuron].set_yticks([])
    axs[2, neuron].set_xlabel("x")
    axs[2, neuron].set_ylim(-2.1, 2.1)



# plot results for each neuron
for neuron in range(1, n_neurons+1):
    axs[0, neuron+1].set_title(f"#neuron: 0.{neuron}")
    axs[0, neuron+1].plot(x, y, color='black')
    for i, y2 in enumerate(ys_separate):
        color = plt.cm.autumn_r(i/n)
        axs[0, neuron+1].plot(x, y2[:,neuron-1], color=color)
    if neuron == 0:
        axs[0, neuron+1].set_ylabel("y")
    else:
        axs[0, neuron+1].set_yticks([])
    axs[0, neuron+1].set_xticks([])
    axs[0, neuron+1].set_ylim(-1.1, 1.1)

    # for i, losses in enumerate(losses_history_separate):
    #     axs[1, neuron+1].plot(x, losses[neuron-1], color=plt.cm.autumn_r(i/n))
    if neuron == 0:
        axs[1, neuron+1].set_ylabel("loss")
    else:
        axs[1, neuron+1].set_yticks([])
    axs[1, neuron+1].set_xticks([])
    axs[1, neuron+1].set_ylim(0, 3)

    axs[2, neuron+1].plot(x, np.zeros_like(x), color='gray', linestyle='--')
    for i, grad in enumerate(grad_history):
        axs[2, neuron+1].plot(x, [g[neuron-1] for g in grad], color=plt.cm.autumn_r(i/n))
    if neuron == 0:
        axs[2, neuron+1].set_ylabel("grad")
    else:
        axs[2, neuron+1].set_yticks([])
    axs[2, neuron+1].set_xlabel("x")
    axs[2, neuron+1].set_ylim(-2.1, 2.1)

plt.tight_layout()
plt.show()

# print(np.array(grad_history).mean(axis=1))